In [81]:
# always get fresh copies!
! cp ~cs186/sp16/hw2/part2/part2.db .
! chmod 700 part2.db
! cp ~cs186/sp16/hw2/part2/part2sampled.db .
! chmod 700 part2sampled.db
! cp ~cs186/sp16/hw2/part2/part2test.db .
! chmod 700 part2test.db

In [82]:
import subprocess, sqlite3, csv, os, sys

# jupyter weirdness; this makes sure that stdout is not buffered
def printNow(s):
    print s
    sys.stdout.flush()

    
def checkFile(f):
    if not (os.path.isfile(f)):
        raise IOError("DB File " + f + " is not here!")


currentDB1 = currentDB2 = "part2sampled.db"

# for grading dataset
# note that the two DB are different! currentDB1 is smaller in size
if ('CS186GRADER' in os.environ):
    currentDB1 = "part2sampled.db" 
    currentDB2 = "part2.db" 
    
checkFile(currentDB1)
dbQ1 = sqlite3.connect(currentDB1) 
dbQ1.text_factory = str

checkFile(currentDB2)
dbQ2 = sqlite3.connect(currentDB2)
dbQ2.text_factory = str

In [88]:
# Q1
# Degrees of separation
# Expected output: a paths table (as defined), note that we will be checking the original
#                  paths, and not just the average value

def initDegrees():
    dbQ1.executescript("""
        -- create paths table
        DROP TABLE IF EXISTS paths;
        DROP TABLE IF EXISTS pathsLastUpdated;
        DROP INDEX IF EXISTS paths_src_idx;
        DROP INDEX IF EXISTS paths_dst_idx;

        CREATE TABLE paths (
            src varchar(200),
            dst varchar(200),
            length integer,
            PRIMARY KEY (src, dst)
        );
        
        CREATE TABLE pathsLastUpdated (
            src varchar(200),
            dst varchar(200),
            length integer
        );

        CREATE INDEX paths_src_idx ON paths(src);
        CREATE INDEX paths_dst_idx ON paths(dst);
    """);
    dbQ1.commit()
    return


def populatePaths():
    dbQ1.executescript("""
      -- initialize paths with the first level connections
      INSERT INTO paths SELECT
        src,
        dst,
        1
      FROM links;      
      
      -- initialized for your convenience the "last udpated"
      INSERT INTO pathsLastUpdated SELECT
        src,
        dst,
        1
      FROM links;
    """);
    
    dbQ1.commit()
    return;


# Please implement the algorithm such that you don't repeat paths that were previously explored
# For example, for graph a->b->c->d, we want to see (a,b), (b,c), (c,d) then (a,c), (b,d), then (a,d)
# This makes a rather significant performance difference. To this end we have set up the 
#   pathsLastUpdated for you to store intermediate results
def getNewPaths():
    dbQ1.executescript("""
        -- this is an intermediate table for you to store temp data
        DROP TABLE IF EXISTS pathsNew;  
        CREATE TABLE pathsNew (
            src varchar(200),
            dst varchar(200),
            length integer
        );    
        
        -- INSERT INTO pathsNew        
            -- TODO (hint: use pathsLastUpdated)
            
            --join (pathsLastUpdated) with (links), finding all the new paths 
            --that extends paths by one hop
            
            -- do you just
            -- go one further from your current dest????
            -- so confused
            -- but avoid cycles????
            -- make sure not in paths --> seen before already?
            -- should be taken care of later??? I think???
            
            
            
            
        INSERT INTO pathsNew
            SELECT p.src, l.dst, p.length + 1
            FROM pathsLastUpdated as p, links as l
            WHERE p.dst = l.src
        ;
            
        -- check both directions??
        INSERT INTO pathsNew
            SELECT l.src, p.dst, p.length + 1
            FROM pathsLastUpdated as p, links as l
            WHERE l.dst = p.src
        ;
        
        -- WAIT SHOOT don't want (a, a, ). Don't go back to yourself --> cycle!!
        
            
        -- ;
     """)
    dbQ1.commit()
    return


# SELECT DISTINCT src,  FROM pathsNew
#             GROUP BY src, dst
#              EXCEPT
#             SELECT * FROM paths

def updatePathsWithNewpaths():
    dbQ1.executescript("""
        DROP TABLE IF EXISTS pathsLastUpdated;
        
        
    
        -- add it to path
        -- INSERT INTO paths
            -- TODO (hint: use pathsNew)
            
            
            --(paths) += (newPaths) that are distinct and new
            
        -- get a table for the entries in pathsNew that would duplicate in paths?
        DROP TABLE IF EXISTS duplicatePaths;
        
        CREATE TABLE duplicatePaths AS
            SELECT pathsNew.src, pathsNew.dst, pathsNew.length
            FROM pathsNew, paths
            WHERE (pathsNew.src = paths.src AND paths.dst = pathsNew.dst)
                OR (pathsNew.src = pathsNew.dst);
        
        
        INSERT INTO paths
            SELECT DISTINCT * FROM pathsNew
            EXCEPT 
            SELECT * FROM duplicatePaths
        
                 
        ;
             
        -- ;
        
        ALTER TABLE pathsNew RENAME TO pathsLastUpdated; 
    """)
    dbQ1.commit()
    return


def getPathsCount():
    r = dbQ1.execute("SELECT COUNT(*) FROM paths;").fetchone()
    return r[0]


def getAveragePathsLength():
    r = dbQ1.execute("SELECT avg(length) FROM paths;").fetchone()
    return r[0]


def findDegreeDistribution():
    printNow("initializing degrees.  Calling sqlite...")
    initDegrees()
    printNow("Done, now populating paths.  Calling sqlite...")
    populatePaths()
    printNow("Done, now getting pathcount.  Calling sqlite...")
    oldCount = getPathsCount()
    
    i = 0
    while True:
        i += 1
        printNow("Beginning iteration #" + str(i) + ".  Calling sqlite ...")
        getNewPaths()
        printNow("Updating paths")
        updatePathsWithNewpaths()
        currentCount = getPathsCount()
        printNow("Done, current path total is: " + str(currentCount))
        if (currentCount == oldCount):
            break
        else:
            oldCount = currentCount

            
    print 'Converged! The average length of the shortest paths is: ', getAveragePathsLength()

findDegreeDistribution()

initializing degrees.  Calling sqlite...
Done, now populating paths.  Calling sqlite...
Done, now getting pathcount.  Calling sqlite...
Beginning iteration #1.  Calling sqlite ...
Updating paths
Done, current path total is: 3817
Beginning iteration #2.  Calling sqlite ...
Updating paths
Done, current path total is: 5542
Beginning iteration #3.  Calling sqlite ...
Updating paths


ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



KeyboardInterrupt: 

In [ ]:
# Q2
# Expected output: nodes table populated with pagerank results 
# Note:
# - keeping track of the old PR value helps find convergence, though it's not required in the
#   assignment, it will help you see how close the values end up becoming.
def initPageRank():
    dbQ2.executescript("""
        DROP TABLE IF EXISTS nodes;
        CREATE TABLE nodes (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
    """);
    dbQ2.commit()
    return


def populatePageRank():
    dbQ2.executescript("""
        INSERT INTO nodes
        SELECT
            DISTINCT links.src,
            1,
            1,
            COUNT(links.dst)
        FROM
            links
        GROUP BY
            links.src;
               
        -- corner case #1 nodes with no outgoing edges
        INSERT INTO
            nodes
        SELECT
            DISTINCT links.dst,
            1,
            1,
            0
        FROM
            links
        WHERE
            links.dst NOT IN (
                SELECT node FROM nodes
            );    
            
        -- corner cases #2 nodes with no incoming edges
        UPDATE
            nodes
        SET
            currentpr = 0.15
        WHERE
            nodes.node not in (
                SELECT DISTINCT dst
                FROM links
            );
    """);
    dbQ2.commit()

    return


def updatePageRank():

    dbQ2.executescript(""" 
        -- TODO (hint: you might want a helper table to store intermediate results)
        
        -- (nodesTemp) = update each node with new pagerank value of 0.15 + 0.85 * (sum of 
        -- incoming edges' pagerank value normalized by their respective source node's outgoing 
        -- degree), using values in (nodes)
        
        
        
        
        DROP TABLE IF EXISTS normalized;
        CREATE TABLE normalized (
            node varchar(200),
            value real,
            nodeDst varchar(200)
        );
        
        -- x% of people from pageA moving to pageB
        -- calculate how much x% actually is as a number?
        -- x% * population(A)
        -- so x% dependent on outdegree(A)
        -- so incoming edge and source node like same!
        
        INSERT INTO normalized
            SELECT links.src, (nodes.currentpr / nodes.outdegree), links.dst 
            FROM links, nodes 
            WHERE links.src = nodes.node
        ;
            
            
        -- THEN CREATE A SUM
        DROP TABLE IF EXISTS sum;
        CREATE TABLE sum (
            node varchar(200),
            value real
        );
        INSERT INTO sum
            SELECT nodeDst, SUM(value)
            FROM normalized
            GROUP BY nodeDst
        ;
            
        
        
        DROP TABLE IF EXISTS nodesTemp;
        CREATE TABLE nodesTemp (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
        INSERT INTO nodesTemp
            SELECT nodes.node, nodes.currentpr, 0.15 + 0.85 * (sum.value), nodes.outdegree
            FROM nodes, sum
            WHERE nodes.node = sum.node
        ;
            
            
        
        
        -- (nodes) gets updated by (nodesTemp)
        DROP TABLE IF EXISTS nodes;
        ALTER TABLE nodesTemp RENAME TO nodes;
        
        

    """);
    dbQ2.commit()

    return

def topPageRank():
    
    dbQ2.executescript("""
    
        DROP VIEW IF EXISTS topPR;
        CREATE VIEW topPR AS
        SELECT *
        FROM nodes
        ORDER BY currentpr DESC
        LIMIT 100;
        
    """);
    dbQ2.commit()

    return

def printTopPageRank():
    # we need to create an cursor object to fetch results to this program
    #    which we didn't need to before since it was just updating the DB
    r = dbQ1.execute("SELECT * FROM topPR;").fetchall()
    for i in r:
        print i
    

def runPageRank(N=10):
    printNow('Initializing page rank')
    initPageRank()
    printNow('Populating PageRank')
    populatePageRank()
    for i in xrange(N):
        printNow('Updating PageRank for iteration {}'.format(i))
        updatePageRank()
        
    printNow('Running top pagerank!')        
    topPageRank()
    printNow('Done! Here are your results')        
    printTopPageRank()
    
    return


runPageRank()

In [ ]:
# you should have ran the two previous cells before running the test!
# make sure also that you have set the db to part2test.db!
! ./test.sh